Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [13]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [14]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [18]:
batch_size = 128
hidden_size = 1024
beta = 5e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    # layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases1 = tf.Variable(tf.zeros([hidden_size]))
    # layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  output1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(output1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ (beta/2)*tf.nn.l2_loss(weights1)+ (beta/2)*tf.nn.l2_loss(weights2)
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_output1 =   tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_output1, weights2) + biases2)
  test_output1 =   tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_output1, weights2) + biases2)



In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # it needs to go back to beginning
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1194.558350
Minibatch accuracy: 7.8%
Validation accuracy: 31.3%
Minibatch loss at step 500: 223.877975
Minibatch accuracy: 86.7%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 63.368038
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 1500: 18.265364
Minibatch accuracy: 92.2%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 5.602058
Minibatch accuracy: 92.2%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 2.040096
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 1.036859
Minibatch accuracy: 85.9%
Validation accuracy: 86.2%
Test accuracy: 92.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0]/200) # it needs to go back to beginning
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_2', defined at:
  File "/Applications/Canopy.app/appdata/canopy-1.7.4.3348.macosx-x86_64/Canopy.app/Contents/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Applications/Canopy.app/appdata/canopy-1.7.4.3348.macosx-x86_64/Canopy.app/Contents/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/miladmahdian/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/miladmahdian/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/miladmahdian/Library/Enthou

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [65]:
batch_size = 128
hidden_size = 1024
beta = 5e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    # layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases1 = tf.Variable(tf.zeros([hidden_size]))
    # layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  keep_prob = tf.placeholder("float")
  output1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1),keep_prob )
  logits = tf.matmul(output1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ (beta/2)*tf.nn.l2_loss(weights1)+ (beta/2)*tf.nn.l2_loss(weights2)
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_output1 =   tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_output1, weights2) + biases2)
  test_output1 =   tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_output1, weights2) + biases2)



In [66]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0]- batch_size) # it needs to go back to beginning
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob :0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1105.105469
Minibatch accuracy: 8.6%
Validation accuracy: 25.5%
Minibatch loss at step 500: 226.746429
Minibatch accuracy: 80.5%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 63.312889
Minibatch accuracy: 80.5%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 18.224081
Minibatch accuracy: 90.6%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 5.581353
Minibatch accuracy: 91.4%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 2.069195
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 1.061207
Minibatch accuracy: 85.2%
Validation accuracy: 85.7%
Test accuracy: 92.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [174]:
#Using Sanitized version

pickle_file = 'notMNIST_san.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset_s = save['train_dataset_san']
  train_labels_s = save['train_labels_san']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset_s.shape, train_labels_s.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (195314, 28, 28) (195314,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [175]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset_s, train_labels_s = reformat(train_dataset_s, train_labels_s)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset_s.shape, train_labels_s.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (195314, 784) (195314, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [26]:
batch_size = 128
hidden_size1 = 1024
hidden_size2 = 512
hidden_size3 = 128


beta = 5e-4

graph = tf.Graph()
def model(data, train=False):
    """The Model definition."""
    output1 = tf.nn.relu(tf.matmul(data, weights1) + biases1)
    if train:
        output1 = tf.nn.dropout(output1, keep_prob)
      
    output2 = tf.nn.relu(tf.matmul(output1, weights2) + biases2)
    if train:
        output2 = tf.nn.dropout(output2, keep_prob)
    output3 = tf.nn.relu(tf.matmul(output2, weights3) + biases3)
    if train:
        output3 = tf.nn.dropout(output3, keep_prob)    
    return tf.matmul(output3, weights4) + biases4

    
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  keep_prob = tf.placeholder("float")

  # layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size1],stddev = 0.1))
  biases1 = tf.Variable(tf.zeros([hidden_size1]))
    # layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size1, hidden_size2],stddev = 0.05))
  biases2 = tf.Variable(tf.zeros([hidden_size2]))
    # layer 3
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_size2, hidden_size3],stddev = 0.025))
  biases3 = tf.Variable(tf.zeros([hidden_size3]))
    # layer 4
  weights4 = tf.Variable(
    tf.truncated_normal([hidden_size3, num_labels],stddev = 0.012))
  biases4 = tf.Variable(tf.zeros([num_labels])) 
    
  #check_op = tf.add_check_numerics_ops()




  
  # Training computation.
  logits = model(tf_train_dataset,True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ (beta/2)*tf.nn.l2_loss(weights1)+ (beta/2)*tf.nn.l2_loss(weights2)+(beta/2)*tf.nn.l2_loss(weights3)+(beta/2)*tf.nn.l2_loss(weights4)
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_steps = 100 # decay every 100 steps
  decay_rate = 0.96 # the exponetial base
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  # Optimizer.
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset) )
  test_prediction = tf.nn.softmax(model(tf_test_dataset) )





In [27]:
num_steps = 7001
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0]) # creates a permutation of index 0 to number of images-1 
  shuffled_dataset = dataset[permutation,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  #step = 0 
  #prev = 0
  #now = -1  
  #while(step<10001 and abs(now - prev)>0.1):
  for step in range(num_steps) :
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0]- batch_size) # it needs to go back to beginning
    if offset<batch_size:
        train_dataset,train_labels = randomize(train_dataset,train_labels)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob :0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #step+=1
    if (step % 500 == 0):
      #prev = now
      #now = accuracy(valid_prediction.eval(), valid_labels)
      print("Minibatch loss at step %d: %f, with learning rate %f" % (step, l,learning_rate.eval()))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)) # best result so far: %94.8


Initialized
Minibatch loss at step 0: 3.222823, with learning rate 0.499796
Minibatch accuracy: 7.8%
Validation accuracy: 10.2%
Minibatch loss at step 500: 1.709558, with learning rate 0.407520
Minibatch accuracy: 76.6%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 1.423056, with learning rate 0.332281
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 1.200069, with learning rate 0.270932
Minibatch accuracy: 83.6%
Validation accuracy: 84.7%
Minibatch loss at step 2000: 1.075927, with learning rate 0.220911
Minibatch accuracy: 82.0%
Validation accuracy: 85.9%
Minibatch loss at step 2500: 1.041256, with learning rate 0.180125
Minibatch accuracy: 79.7%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 0.917623, with learning rate 0.146869
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 0.704685, with learning rate 0.119753
Minibatch accuracy: 90.6%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.80